Course Instructor: Bernd Neumayr, JKU

# UE05: SHACL

Complete the **10 tasks (1 point per task)** in the `4. SHACL` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- The data graph and your solution (the shapes graph) in executable form
- After executing the validation, print out the validation results in tabular form.  


Vorbereitungen und Inports: 

In [6]:
# Install required packages in the current Jupyter kernel
!pip install -q rdflib 
!pip3 install -q pyshacl

# Imports
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
from rdflib.namespace import FOAF , XSD , RDFS 
from rdflib.plugins.sparql.processor import SPARQLResult
from rdflib.namespace import NamespaceManager

from pyshacl import validate

import pandas as pd

def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = [ { var : res[var].n3(graph.namespace_manager) if (isinstance(res[var],URIRef) and use_prefixes) else res[var] for var in results.vars } for res in results ]     
                                        # construct a list of dictionaries, as intermediate format to construct the pandas DataFrame, use prefixes to abbreviate URIs                
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def validation_report_as_dataframe(validation_report):
  df = sparql_select(results_graph,"""
		SELECT  ?focusNode ?resultPath ?value ?sourceConstraintComponent ?sourceShape ?resultMessage
		WHERE
  		{ ?vr	a sh:ValidationResult ;
						sh:focusNode ?focusNode ;
						sh:sourceConstraintComponent ?sourceConstraintComponent ;
						sh:sourceShape ?sourceShape ;
						sh:resultMessage ?resultMessage .					 
				OPTIONAL { ?vr sh:value ?value . }
				OPTIONAL { ?vr sh:resultPath ?resultPath . }
  		}
  """,use_prefixes=True)
  return df

def shacl_validate(dg,sg):
  return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)  
  

def shacl_validate_with_rules(dg,sg):
	return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=True,
      iterate_rules=True, inplace=True,
      js=False,
      debug=False)

**Task 1:**

Wenn eine Person eine x:knows Beziehung hat, dann sollte das Ziel dieser Beziehung als IRI angegeben sein.

In [22]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Peter>  rdf:type  <Person> ;
        :knows    <John> ;
        :knows    [ :name  "Peter" ] .

<John>  :knows  [ :name   "Mary" ] .     
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonShape> a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property[
    sh:path :knows;
    sh:nodeKind sh:IRI].


""")



conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value is not of Node Kind sh:IRI" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape [ sh:nodeKind sh:IRI ;
                    sh:path :knows ] ;
            sh:value [ a rdfs:Resource ;
                    :name "Peter" ] ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Peter>,:knows,n9ea55f4172064636a8c0b8e19e1ef5d2b1,sh:NodeKindConstraintComponent,n6291266699634e5c8066ec6e85068f6eb1,Value is not of Node Kind sh:IRI


**Task 2:**

Die Klassen Man und Woman sind disjunkt. Hinweis: verwenden Sie sh:not.


In [8]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<YoungMan>  rdfs:subClassOf  <Man> .

<John>  rdf:type  <YoungWoman> , <YoungMan> .

<Mary>  rdf:type  <Woman> .

<Peter>  rdf:type  <Man> .

<YoungWoman>  rdfs:subClassOf  <Woman> .

<Jane>  rdf:type  <YoungWoman> .

<Susi>  rdf:type  <YoungWoman> , <Man> .    
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<GenderShape>  a  sh:NodeShape ;
    sh:targetSubjectsOf rdf:type;
    sh:not[
      sh:and(
        [sh:class <Man>]
        [sh:class <Woman>]
    )].
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/John> ;
            sh:resultMessage "Node <http://example.org/entities/John> conforms to shape [ sh:and ( [ sh:class <http://example.org/entities/Man> ] [ sh:class <http://example.org/entities/Woman> ] ) ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NotConstraintComponent ;
            sh:sourceShape <http://example.org/entities/GenderShape> ;
            sh:value <http://example.org/entities/John> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Susi> ;
            sh:resultMessage "Node <http://example.org/entities/Susi> conforms to shape [ sh:and ( [ sh:class <http://example.org/entities/Man> ] [ sh:class <http://example.org/entities/Woman> ] ) ]" ;
       

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Susi>,None,<http://example.org/entities/Susi>,sh:NotConstraintComponent,<http://example.org/entities/GenderShape>,Node <http://example.org/entities/Susi> confor...
1,<http://example.org/entities/John>,None,<http://example.org/entities/John>,sh:NotConstraintComponent,<http://example.org/entities/GenderShape>,Node <http://example.org/entities/John> confor...


**Task 3:**

Wenn eine Person eine :knows Beziehung hat, dann muss das Ziel dieser Beziehung eine Person sein.

In [9]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Peter>  rdf:type  <Person> ;
        :knows    <Mary> ;
        :knows    [ :name  "Mary" ] ;
        :knows    [ rdf:type  <Person> ] .

<Jane>  :knows  <Jim> .   
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonShape> a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonKnowsShape> .

<PersonKnowsShape> a sh:PropertyShape ;
    sh:path :knows ;
    sh:class <Person> .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Person>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape <http://example.org/entities/PersonKnowsShape> ;
            sh:value <http://example.org/entities/Mary> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Person>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violat

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Mary>,sh:ClassConstraintComponent,<http://example.org/entities/PersonKnowsShape>,Value does not have class <http://example.org/...
1,<http://example.org/entities/Peter>,:knows,n172f044328b545cf90e2de05948595bbb1,sh:ClassConstraintComponent,<http://example.org/entities/PersonKnowsShape>,Value does not have class <http://example.org/...


**Task 4:**

Jede Person hat genau ein Alter (:age). Eine Person ist mindestens 0 und maximal 150 Jahre alt. Erwachsene (Klasse Adult) sind mindestens 19 Jahre alt. Senioren sind mindestens 65 Jahre alt. Kinder sind maximal 12 Jahre alt.
Verwenden Sie sh:minInclusive und sh:maxInclusive.

In [10]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<ABC>   :age    -1 , 27 , 197 .

<John>  rdf:type  <Person> ;
        :age      151 .

<Adult>  rdfs:subClassOf  <Person> .

<Beny>  rdf:type  <Child> ;
        :age      13 .

<Zoe>   rdf:type  <Child> ;
        :age      12 .

<Mary>  rdf:type  <Woman> ;
        :age      17 .

<Bibi>  rdf:type  <Person> ;
        :age      0 .

<Peter>  rdf:type  <Person> .

<Senior>  rdfs:subClassOf  <Adult> .

<Jane>  rdf:type  <Woman> ;
        :age      67 .

<Lili>  rdf:type  <Person> ;
        :age      -1 .

<Senior1>  rdf:type  <Senior> ;
        :age      65 .

<Child>  rdfs:subClassOf  <Person> .

<Bibo>  rdf:type  <Person> ;
        :age      34 , 12 .

<Woman>  rdfs:subClassOf  <Adult> .

<Senior2>  rdf:type  <Senior> ;
        :age      64 .   
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonShape> a sh:NodeShape;
    sh:targetClass <Person>;
    sh:property[
    sh:path :age;
    sh:maxCount 1;
    sh:minCount 1;
    sh:minInclusive 0;
    sh:maxInclusive 150].

<AdultShape> a sh:NodeShape ;
    sh:targetClass <Adult> ;
    sh:property [
    sh:path :age;
    sh:minInclusive 19].

<SeniorShape> a sh:NodeShape;
    sh:targetClass <Senior>;
    sh:property[
    sh:path :age; 
    sh:minInclusive 65].

<ChildShape> a sh:NodeShape; 
    sh:targetClass <Child>;
    sh:property[
    sh:path :age; 
    sh:maxInclusive 12].
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Lili> ;
            sh:resultMessage "Value is not >= Literal(\"0\", datatype=xsd:integer)" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MinInclusiveConstraintComponent ;
            sh:sourceShape _:nb19282e28e634c1eb1986e3d142b78a0b1 ;
            sh:value -1 ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Senior2> ;
            sh:resultMessage "Value is not >= Literal(\"65\", datatype=xsd:integer)" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MinInclusiveConstraintComponent ;
            sh:sourceShape [

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Lili>,:age,-1,sh:MinInclusiveConstraintComponent,nb19282e28e634c1eb1986e3d142b78a0b1,"Value is not >= Literal(""0"", datatype=xsd:inte..."
1,<http://example.org/entities/Bibo>,:age,None,sh:MaxCountConstraintComponent,nb19282e28e634c1eb1986e3d142b78a0b1,More than 1 values on <http://example.org/enti...
2,<http://example.org/entities/Peter>,:age,None,sh:MinCountConstraintComponent,nb19282e28e634c1eb1986e3d142b78a0b1,Less than 1 values on <http://example.org/enti...
3,<http://example.org/entities/John>,:age,151,sh:MaxInclusiveConstraintComponent,nb19282e28e634c1eb1986e3d142b78a0b1,"Value is not <= Literal(""150"", datatype=xsd:in..."
4,<http://example.org/entities/Beny>,:age,13,sh:MaxInclusiveConstraintComponent,nb19282e28e634c1eb1986e3d142b78a0b4,"Value is not <= Literal(""12"", datatype=xsd:int..."
5,<http://example.org/entities/Senior2>,:age,64,sh:MinInclusiveConstraintComponent,nb19282e28e634c1eb1986e3d142b78a0b3,"Value is not >= Literal(""65"", datatype=xsd:int..."
6,<http://example.org/entities/Mary>,:age,17,sh:MinInclusiveConstraintComponent,nb19282e28e634c1eb1986e3d142b78a0b2,"Value is not >= Literal(""19"", datatype=xsd:int..."


**Task 5:**

Jede Person hat maximal einen Namen. Der Name muss ein Literal sein und vom Datentyp String. Personen dürfen abgesehen von :name und rdf:type keine weiteren Eigenschaften haben.

In [11]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<John>  rdf:type  <MalePerson> ;
        :knows    <Mary> .

<FemalePerson>  rdfs:subClassOf  <Person> .

<Mary>  rdf:type  <FemalePerson> ;
        :name     "Mary" .

<Bibi>  rdf:type  <Person> ;
        :name     "Jim" , "Bibi" .

<Jane>  rdf:type  <Person> ;
        :name     2343 .

<Jim>   rdf:type  <Person> ;
        :name     <Jim> .

<MalePerson>  rdfs:subClassOf  <Person> .   
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonShape> a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonPropertyShape>;
    sh:closed true;
    sh:ignoredProperties (rdf:type) .

<PersonPropertyShape> a sh:PropertyShape;
    sh:path :name;
    sh:maxCount 1;
    sh:datatype xsd:string;
    sh:nodeKind sh:Literal.
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Jane> ;
            sh:resultMessage "Value is not Literal with datatype xsd:string" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:DatatypeConstraintComponent ;
            sh:sourceShape <http://example.org/entities/PersonPropertyShape> ;
            sh:value 2343 ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/John> ;
            sh:resultMessage "Node <http://example.org/entities/John> is closed. It cannot have value: <http://example.org/entities/Mary>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:Close

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Jane>,:name,2343,sh:DatatypeConstraintComponent,<http://example.org/entities/PersonPropertyShape>,Value is not Literal with datatype xsd:string
1,<http://example.org/entities/Jim>,:name,<http://example.org/entities/Jim>,sh:DatatypeConstraintComponent,<http://example.org/entities/PersonPropertyShape>,Value is not Literal with datatype xsd:string
2,<http://example.org/entities/Jim>,:name,<http://example.org/entities/Jim>,sh:NodeKindConstraintComponent,<http://example.org/entities/PersonPropertyShape>,Value is not of Node Kind sh:Literal
3,<http://example.org/entities/Bibi>,:name,None,sh:MaxCountConstraintComponent,<http://example.org/entities/PersonPropertyShape>,More than 1 values on <http://example.org/enti...
4,<http://example.org/entities/John>,:knows,<http://example.org/entities/Mary>,sh:ClosedConstraintComponent,<http://example.org/entities/PersonShape>,Node <http://example.org/entities/John> is clo...


**Task 6:**

Städte sind via Property :inCountry Ländern zugeordnet. Europäische Städte sind europäischen Ländern zugeordnet. Österreichische Städte sind Austria zugeordnet.

In [12]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Paris>  rdf:type   <EuropeanCity> ;
        :inCountry  <France> .

<AustrianCity>  rdfs:subClassOf  <EuropeanCity> .

<Salzburg>  rdf:type  <AustrianCity> ;
        :inCountry  <Germany> .

<EuropeanCountry>  rdfs:subClassOf  <Country> .

<NewYork>  rdf:type  <City> ;
        :inCountry  <USA> .

<Germany>  rdf:type  <EuropeanCountry> .

<EuropeanCity>  rdfs:subClassOf  <City> .

<Vienna>  rdf:type  <AustrianCity> ;
        :inCountry  <Austria> .

<Austria>  rdf:type  <EuropeanCountry> .  
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<CityInCountryShape> a sh:NodeShape;
    sh:targetSubjectsOf rdf:type;
    sh:property[
    sh:path :inCountry;
    sh:class <Country>].

<EuropeanCityInCountryShape> a sh:NodeShape;
    sh:targetClass <EuropeanCity>;
    sh:property[
    sh:path :inCountry;
    sh:class <EuropeanCountry>].

<AustrianCityInCountryShape> a sh:NodeShape;
    sh:targetClass <AustrianCity>;
    sh:property[
    sh:path :inCountry;
    sh:in (<Austria>)].
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Paris> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Country>" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape _:n62b789efa7d8404db3039303b9c2e925b1 ;
            sh:value <http://example.org/entities/France> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Paris> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/EuropeanCountry>" ;
            sh:resultPath :inCountry ;
            sh:resultSe

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/NewYork>,:inCountry,<http://example.org/entities/USA>,sh:ClassConstraintComponent,n62b789efa7d8404db3039303b9c2e925b1,Value does not have class <http://example.org/...
1,<http://example.org/entities/Paris>,:inCountry,<http://example.org/entities/France>,sh:ClassConstraintComponent,n62b789efa7d8404db3039303b9c2e925b1,Value does not have class <http://example.org/...
2,<http://example.org/entities/Salzburg>,:inCountry,<http://example.org/entities/Germany>,sh:InConstraintComponent,n62b789efa7d8404db3039303b9c2e925b3,Value <http://example.org/entities/Germany> no...
3,<http://example.org/entities/Paris>,:inCountry,<http://example.org/entities/France>,sh:ClassConstraintComponent,n62b789efa7d8404db3039303b9c2e925b2,Value does not have class <http://example.org/...


**Task 7:**

Die Property :worksFor darf nur Personen (als Subjekt) und Organisation (als Objekt) miteinander verbinden.
Hinweis: Lösen Sie die Aufgabe ohne Property Shapes.


In [13]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Mary>  :worksFor  <JKU> .

<Jane>  rdf:type   <Person> ;
        :worksFor  <LMU> .

<Jim>   rdf:type   <Man> ;
        :worksFor  <LMU> .

<University>  rdfs:subClassOf  <Organization> .

<Bob>   rdf:type   <Person> ;
        :worksFor  [ rdf:type  <Organization> ] .

<JKU>   rdf:type  <University> .   
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<WorksForObjectShape> a sh:NodeShape;
    sh:targetObjectsOf :worksFor;
    sh:class <Organization>.

<WorksForSubjectShape> a sh:NodeShape;
    sh:targetSubjectsOf :worksFor;
    sh:class <Person>.
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Mary> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Person>" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape <http://example.org/entities/WorksForSubjectShape> ;
            sh:value <http://example.org/entities/Mary> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/LMU> ;
            sh:resultMessage "Value does not have class <http://example.org/entities/Organization>" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape <http://example.org/entities/WorksForObjectShape> ;
        

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/LMU>,None,<http://example.org/entities/LMU>,sh:ClassConstraintComponent,<http://example.org/entities/WorksForObjectShape>,Value does not have class <http://example.org/...
1,<http://example.org/entities/Mary>,None,<http://example.org/entities/Mary>,sh:ClassConstraintComponent,<http://example.org/entities/WorksForSubjectSh...,Value does not have class <http://example.org/...
2,<http://example.org/entities/Jim>,None,<http://example.org/entities/Jim>,sh:ClassConstraintComponent,<http://example.org/entities/WorksForSubjectSh...,Value does not have class <http://example.org/...


**Task 8:**

Peter kennt ausschließlich Männer, die in Österreich leben.

In [14]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<John>  rdf:type  <Man> ;
        :livesIn  <Austria> .

<Mary>  rdf:type  <Woman> ;
        :knows    <Jim> ;
        :livesIn  <Austria> .

<Hans>  rdf:type  <Man> .

<Peter>  :knows  <Josef> , <Hans> , <Franz> , <John> , <Mary> .

<Franz>  rdf:type  <Man> ;
        :livesIn  <Germany> . 
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PeterShape> a sh:NodeShape;
    sh:targetNode <Peter>;
    sh:closed true;
    sh:property [
    sh:path :knows;
    sh:node <PeterKnowsShape>].

<PeterKnowsShape> a sh:NodeShape;
    sh:class <Man>;
    sh:property[
    sh:path :livesIn;
    sh:class <Austria>].
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not conform to Shape <http://example.org/entities/PeterKnowsShape>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeConstraintComponent ;
            sh:sourceShape _:n9868ad28a59246dea15c8f2a4c8116fcb1 ;
            sh:value <http://example.org/entities/John> ],
        [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Peter> ;
            sh:resultMessage "Value does not conform to Shape <http://example.org/entities/PeterKnowsShape>" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstrain

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Mary>,sh:NodeConstraintComponent,n9868ad28a59246dea15c8f2a4c8116fcb1,Value does not conform to Shape <http://exampl...
1,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Franz>,sh:NodeConstraintComponent,n9868ad28a59246dea15c8f2a4c8116fcb1,Value does not conform to Shape <http://exampl...
2,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/Josef>,sh:NodeConstraintComponent,n9868ad28a59246dea15c8f2a4c8116fcb1,Value does not conform to Shape <http://exampl...
3,<http://example.org/entities/Peter>,:knows,<http://example.org/entities/John>,sh:NodeConstraintComponent,n9868ad28a59246dea15c8f2a4c8116fcb1,Value does not conform to Shape <http://exampl...


**Task 9:**

Das Nettogewicht eines Produkts darf nicht größer sein als das Bruttogewicht.
Verwenden Sie sh:lessThanOrEquals.


In [15]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Cookies>  rdf:type   <Product> ;
        :grossWeight  0.2 ;
        :netWeight    0.12 .

<Milk>  rdf:type      <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1 .

<Peter>  rdf:type     <Person> ;
        :grossWeight  74 ;
        :netWeight    72 .

<Bread>  rdf:type     <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1.2 .  
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<ProductShape> a sh:NodeShape;
    sh:targetClass <Product>;
    sh:property[
    sh:path :grossWeight];
    sh:property[
    sh:path :netWeight;
    sh:lessThanOrEquals :grossWeight].
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/properties/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Bread> ;
            sh:resultMessage "Value of <http://example.org/entities/Bread>->:grossWeight < Literal(\"1.2\", datatype=xsd:decimal)" ;
            sh:resultPath :netWeight ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:LessThanOrEqualsConstraintComponent ;
            sh:sourceShape [ sh:lessThanOrEquals :grossWeight ;
                    sh:path :netWeight ] ;
            sh:value 1.2 ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Bread>,:netWeight,1.2,sh:LessThanOrEqualsConstraintComponent,n054311e98fe94c2bb892bf6158f87317b2,Value of <http://example.org/entities/Bread>->...


**Task 10:**

Die Objekte von knows-Statements haben einen Namen (Property :name) oder haben einen Vornamen (:givenName) und einen Nachnamen (:familyName). 

In [16]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<Sara>  :givenName  "Sarah" .

<John>  :familyName  "Black" .

<Mary>  :knows  <Pete> .

<Bob>   :familyName  "Builder" ;
        :givenName   "Bob" .

<Pete>  :knows  <Bob> , <Sara> , <John> ;
        :name   "Peter Parker" .   
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/properties/>
BASE <http://example.org/entities/>

<PersonShape> a sh:NodeShape;
    sh:targetObjectsOf :knows; 
    sh:or(
      [sh:and(
        [sh:path :familyName; 
        sh:minCount 1;
        sh:maxCount 1]
        [sh:path :givenName;
        sh:minCount 1;
        sh:maxCount 1])]
      [sh:path :name;
      sh:minCount 1;
      sh:maxCount 1]).
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <http://example.org/entities/Sara> ;
            sh:resultMessage "Node <http://example.org/entities/Sara> does not conform to one or more shapes in [ sh:and ( [ sh:maxCount Literal(\"1\", datatype=xsd:integer) ; sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :familyName ] [ sh:maxCount Literal(\"1\", datatype=xsd:integer) ; sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :givenName ] ) ] , [ sh:maxCount Literal(\"1\", datatype=xsd:integer) ; sh:minCount Literal(\"1\", datatype=xsd:integer) ; sh:path :name ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:OrConstraintComponent ;
            sh:sourceShape <http://example.org/entities/PersonShape> ;
            sh:value <http://example.org/entities/Sara> ],
     

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,<http://example.org/entities/Sara>,None,<http://example.org/entities/Sara>,sh:OrConstraintComponent,<http://example.org/entities/PersonShape>,Node <http://example.org/entities/Sara> does n...
1,<http://example.org/entities/John>,None,<http://example.org/entities/John>,sh:OrConstraintComponent,<http://example.org/entities/PersonShape>,Node <http://example.org/entities/John> does n...
